Run all cells under the Imports and Functions headers first: 

*   Click the arrow to run all cells under header
*   Or hit shift + enter for each and every individual cell.

# Imports

In [23]:
# xmltodict is not a standard Colab module, so need to download and install it every session.
try:
    import xmltodict
    print("module 'xmltodict' is installed")
except ModuleNotFoundError:
  !pip install xmltodict

In [24]:
from sklearn.linear_model import LassoCV, RidgeCV, HuberRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
# import statsmodels.api as sm
from functools import reduce

import requests
import xmltodict
import datetime

# Functions

## Helper Functions

In [102]:
# Web Call to Access and Download Data of a Single Station from AWDB Web Service (SOAP API)

def soap_call(stationtriplets, elementCD, begindate, enddate):
  global xml, dict_of_xml, df
  # Create a dictionaries to store the data
  headers = {'Content-type': 'text/soap'}
  # current_dictionary = {}
  
  # Define Web Service URL
  URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

  # Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
  SOAP_current = '''
  <?xml version="1.0" encoding="UTF-8"?>
  <SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <SOAP-ENV:Body>
      <q0:getData>
        <stationTriplets>STATIONTRIPLETS</stationTriplets>
        <elementCd>ELEMENTCD</elementCd>   
        <ordinal>1</ordinal>
        <duration>DAILY</duration>
        <getFlags>false</getFlags>
        <beginDate>BEGINDATE</beginDate>
        <endDate>ENDDATE</endDate>
        <alwaysReturnDailyFeb29>false</alwaysReturnDailyFeb29>   
      </q0:getData>
    </SOAP-ENV:Body>
  </SOAP-ENV:Envelope>

  '''.strip()
  #Read GetData documents - If <alwaysReturnDailyFeb29> is set to true, will set a null for every non leap year on the 29th,  
  #which breaks this request when selecting date ranges that include Feb 29.
  #Possible element codes: PREC, WTEQ (Water Equivalent/SWE)
  
  # Post SOAP Elements to AWDB Web Service and process results - getData
  SOAP_current = SOAP_current.replace("ELEMENTCD", elementCD)
  SOAP_current = SOAP_current.replace("STATIONTRIPLETS", stationtriplets)
  SOAP_current = SOAP_current.replace("BEGINDATE", begindate)
  SOAP_current = SOAP_current.replace("ENDDATE", enddate)

  #Send request to server and receive xml document
  xml = requests.post(URL, data=SOAP_current, headers=headers)

  #convert xml document to a dictionary, extract values putting them in a dataframe.  XML's aren't the easiest to parse and extract data from, so this is a nice work around.
  dict_of_xml = xmltodict.parse(xml.text)
  df = dict_of_xml['soap:Envelope']['soap:Body']['ns2:getDataResponse']['return']['values']

  #Null values are given as OrderedDictionaries with lots of text, while actual values are given as strings.  This converts all the OrderedDictionaries into actual null/none values, and converts all values that were given as strings into float numbers.
  df = pd.DataFrame(map(lambda i: float(i) if type(i) == str else None, df))

  #Since invidual dates aren't associated with the values in the xml document, have to create a range of dates bw the begindate and endate, which is then added to the dataframe.
  df['Date'] = pd.date_range(begindate,enddate,freq='d')
  df.columns = [f'{elementCD}','Date']
  df.set_index('Date', inplace=True)

  return df


In [103]:
# Function to Download Multiple Stations at a time from AWDB Web Service

def getData(stations, parameter_of_interest, begindate, enddate):
  data_singleDF = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [soap_call(stationtriplets=j,elementCD=parameter_of_interest,begindate=begindate,enddate=enddate) for j in stations])
  data_singleDF.columns = [f'{j}' for j in stations]
  return data_singleDF

In [27]:
#Function that assigns the regression model to be used and its parameters

def regressionModel(regression_model):
  if regression_model == 'Lasso':
    regr = LassoCV(alphas=(0.001,0.01,0.1,1,10,100,1000))
    return regr
  elif regression_model == 'Ridge':
    regr = RidgeCV(alphas=(0.001,0.01,0.1,1,10,100,1000))
    return regr
  elif regression_model == 'SVM':  #Support vector machines
    regr = svm.SVR(kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
    return regr
  elif regression_model == 'Huber':
    regr = HuberRegressor()
    return regr
  elif regression_model == 'GradientBoost':
    regr = GradientBoostingRegressor(random_state=0, n_estimators=100)
    return regr
  elif regression_model == 'AdaBoost':
    regr = AdaBoostRegressor(random_state=0, n_estimators=100)
    return regr
  elif regression_model == 'Random Forest':
    regr = RandomForestRegressor(n_estimators=100)
    return regr
  else:
    print('Choose either Lasso, Ridge, SVM, Huber, GradientBoost, AdaBoost, or Random Forest')

## Regression Functions

### RegressionFun Class

In [104]:
class RegressionFun():

    '''
    stations: list of stationtriplets, with the response station in the first position followed by the predictor station(s).  Ex. ['817:WA:SNTL', '711:WA:SNTL', '975:WA:SNTL']
    parameter_of_interest: can choose 'WTEQ', 'SNWD', 'PREC', 'PRCP', 'TAVG', 'TOBS', and others (https://www.nrcs.usda.gov/wps/portal/wcc/home/dataAccessHelp/webService/webServiceReference/!ut/p/z1/jc9NDoIwEAXgs3CCPqCWuqxNBFIiqdCAsyFdkSaKLozn1xg2LmyY3STfmx9GbGS0-FeY_TPcF3_99BcSk5Sapy1HU576HFactTtWJkMKNnxBnUlVyQ6m1JxD7XXRN10BcwCjLXn8KbUxHwEUHz8w-l0BWWew2hre7pBDpyuIvRgFvVhB5IrHzbkRoZ5VkrwBnC-XyQ!!/?1dmy&current=true&urile=wcm%3apath%3a%2Fwcc%2Bcontent%2Fhome%2Fdata%2Baccess%2Bhelp%2Fweb%2Bservice%2Fweb%2Bservice%2Breference#elementCodes).
    '''

  def __init__(self, stations, parameter_of_interest): #, stations, parameter_of_interest):

    self.stations = stations
    self.parameter_of_interest = parameter_of_interest
    
  def check_model(self, begindate, enddate, regression_model, test_size):
    
    '''
    Function checks model fit on train and test sets.  Use to check which stations result in the best fitting model.  Once the best model is found, it can be used in the make_predictions function to predict null values.
    
    begindate: non-null date in 'mm/dd/yyyy' format
    enddate: non-null date in 'mm/dd/yyyy' format
    regression_model: can be 'Ridge', 'Lasso', 'Huber', 'SVM', 'Random Forest', 'AdaBoost', 'GradientBoost'
    test_size: value between 0 and 1 that defines the percentage of the data to be used as the test_size
    '''

    #Download Data from AWDB Web Service
    data = getData(self.stations, self.parameter_of_interest, begindate, enddate).dropna()

    #Define Targets and Features (e.g. Response and Predictor Variables)
    target = data.iloc[:,0]
    features = data.iloc[:,1:]

    #Split into training and test sets
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=test_size, shuffle=False)
  
    #Choose Regression Model:
    regr = regressionModel(regression_model)
  
    #Fit model on training set
    regr.fit(features_train, target_train)

    self.regr = regr

    #Run predictions on training features and test features
    target_train_pred = regr.predict(features_train)
    target_test_pred = regr.predict(features_test)

    #Print Root Mean Square Error for training and test sets:
    print('RMSE for training set', mean_squared_error(target_train, target_train_pred))
    print('RMSE for test set', mean_squared_error(target_test, target_test_pred))

    #Predictions plot

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y = target_train,
        x = target_train.index,
        mode = 'lines',
        name = 'Training Data'
    ))

    fig.add_trace(go.Scatter(
        y = target_train_pred,
        x = target_train.index,
        mode = 'lines',
        name = 'Model Predictions on Training Data'
    ))

    fig.add_trace(go.Scatter(
        y = target_test,
        x = target_test.index,
        mode = 'lines',
        name = 'Test Data'
    ))

    fig.add_trace(go.Scatter(
        y = target_test_pred,
        x = target_test.index,
        mode = 'lines',
        name = 'Model Predictions on Test Data'
    ))

    fig.update_yaxes(title_text = f"{self.parameter_of_interest}")
    fig.update_xaxes(title_text = 'Date') 

    fig.update_layout(
        showlegend=True,
        height=450,
        width=1200 
    )
    fig.show()

    # # # # # Regression Plot # # # # #

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
      x=features_train.iloc[:,0],
      y=target_train,
      mode='markers',
      hovertext = features_train.index,
      name =  'Response vs. Predictors'
    ))

    fig2.add_trace(go.Scatter(
      x=features_train.iloc[:,0],
      y=pd.DataFrame(target_train_pred.tolist()).iloc[:,0],
      mode='lines',
      hovertext = features_train.index,
      name = 'Model Fit'  
    ))
    
    fig2.update_yaxes(title_text = f'{self.stations[1:]} {self.parameter_of_interest}')
    fig2.update_xaxes(title_text = f'{self.stations[0]} {self.parameter_of_interest}')
    fig2.update_layout(
        showlegend=True,
        height=450,
        width=1200 
    )
    fig2.show()

    return regr
      
      
  def make_predictions(self, predict_begindate, predict_enddate):

    #Download Data from AWDB Web Service
    predict_data  = getData(self.stations, self.parameter_of_interest, predict_begindate, predict_enddate)

    predict_target = predict_data.iloc[:,0].fillna(0)
    predict_features = predict_data.iloc[:,1:].fillna(0)

    #Run predictions
    predictions = self.regr.predict(predict_features)
    
    #Plot predictions 
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        y = predict_target,
        x = predict_target.index,
        mode = 'lines',
        name = f'{self.stations[0]} {self.parameter_of_interest}'
    )) 

    fig.add_trace(go.Scatter(
        y = predictions,
        x = predict_features.index,
        mode = 'lines',
        name = 'Model Predictions'
    )) 

    fig.update_layout(
      showlegend=True,
      height=650,
      width=1400,
      title={
          'text': "Model Predictions",
          'xanchor': 'center',
          'yanchor': 'top',
          'y':0.9,
          'x':0.4},
      xaxis_title = "Date",
      yaxis_title = f"{self.parameter_of_interest} (in)"
    )

    fig.show()  

IndentationError: ignored

### Other functions: Keep for now

In [ ]:
def Predictions(stations, parameter_of_interest, train_begindate, train_enddate, predict_begindate, predict_enddate, regression_model):

  # global predict_target, predict_features, train_target, train_features

  #Download Data from AWDB Web Service
  train_data = getData(stations, parameter_of_interest, train_begindate, train_enddate)
  predict_data  = getData(stations, parameter_of_interest, predict_begindate, predict_enddate)
  #Define Targets and Features (e.g. Dependent and Independent Variables)
  train_target = train_data.iloc[:,0].fillna(0)
  train_features = train_data.iloc[:,1:].fillna(0)

  predict_target = predict_data.iloc[:,0].fillna(0)
  predict_features = predict_data.iloc[:,1:].fillna(0)

  # Choose Regression Model:
  regr = regressionModel(regression_model)

  #Fit model to training set
  regr.fit(train_features, train_target)

  #Run predictions
  predictions = regr.predict(predict_features)
  
  #Plot predictions 
  fig = go.Figure()

  fig.add_trace(go.Scatter(
      y = predict_target,
      x = predict_target.index,
      mode = 'lines',
      name = f'{stations[0]} {parameter_of_interest}'
  )) 

  fig.add_trace(go.Scatter(
      y = predictions,
      x = predict_features.index,
      mode = 'lines',
      name = 'Model Predictions'
  )) 

  fig.update_layout(
    showlegend=True,
    height=650,
    width=1400,
    title={
        'text': "Model Predictions",
        'xanchor': 'center',
        'yanchor': 'top',
        'y':0.9,
        'x':0.4},
    xaxis_title = "Date",
    yaxis_title = f"{parameter_of_interest} (in)"
  )

  fig.show()

# Estimating Missing Data

In [105]:
Blewett = RegressionFun(['352:WA:SNTL','507:WA:SNTL'], 'WTEQ')
Blewett.check_model('10/01/2020','02/26/2021','Ridge',0.2)

RMSE for training set 0.05394207508293898
RMSE for test set 0.7671892167746466


RidgeCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=False)

In [76]:
Blewett.make_predictions('01/01/2021','05/01/2021')

In [81]:
#Aneroid Lake

Aneroid = RegressionFun(['302:OR:SNTL', '653:OR:SNTL'],'WTEQ')

In [82]:
Aneroid.check_model('12/01/2019', '11/29/2020', 'Ridge', 0.2)

RMSE for training set 13.638555594614834
RMSE for test set 4.893397037554024


RidgeCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=False)

In [83]:
Aneroid.make_predictions('11/01/2020','05/01/2021')

In [101]:
#Morse Lake Predictions

Morse = RegressionFun(['642:WA:SNTL', '1085:WA:SNTL', '679:WA:SNTL'], 'WTEQ')
Morse.check_model('10/01/2020', '12/22/2020','Ridge',0.2)

RMSE for training set 0.7233212621481695
RMSE for test set 2.454167025685121


RidgeCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=False)

In [87]:
Morse.make_predictions('12/22/2020', '05/01/2021')

In [88]:
 #Mt. Crag Predictions

crag = RegressionFun(['648:WA:SNTL', '1107:WA:SNTL','974:WA:SNTL'], 'WTEQ')
crag.check_model('11/10/2019','12/10/2020','Ridge',0.2)

RMSE for training set 7.6478199672806815
RMSE for test set 0.984479243957287


RidgeCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=False)

In [89]:
crag.make_predictions('01/01/2021','05/01/2021')

In [91]:
 #Olallie Meadows Predictions

Olallie = RegressionFun(['672:WA:SNTL', '788:WA:SNTL', '898:WA:SNTL'],'WTEQ')
Olallie.check_model('01/01/2021','03/31/2021','Ridge',0.2)

RMSE for training set 1.101389303932015
RMSE for test set 1.517951406146678


RidgeCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=False)

In [92]:
Olallie.make_predictions('01/01/2021', '05/01/2021')

In [97]:
 #Thunder Basin Predictions
Thunder = RegressionFun(['817:WA:SNTL', '711:WA:SNTL'],'WTEQ')  #975 pulls SWE down while 681 greatly pushes it up.  711 slightly raises it.
Thunder.check_model('01/01/2021', '02/28/2021', 'Ridge', 0.2)

RMSE for training set 0.30442821244857127
RMSE for test set 0.20821906785177993


RidgeCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=False)

In [96]:
Thunder.make_predictions('01/01/2021', '05/01/2021')